# Ex06 - Numba CPU/GPU

O objetivo deste exercício é averiguar os ganhos de desempenho do emprego de Numba.
- Parte 1: ver o efeito da compilação JIT (*just-in-time*), 
- Parte 2: ver o efeito da compilação para código a ser executado na GPU
  - Utilize um *runtime* GPU

---



In [113]:
!pip install numba

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Parte 1 - JIT para CPU

O código abaixo implementa o cálculo aproximado do número de pi através do método de monte-carlo. Para uma quantidade `n` de vezes, a função gera dois números pseudoaleatórios entre 0 e 1 com `uniform` que representam uma coordenada no espaço cartesiano compreendido entre 0 e 1 tanto no eixo x quanto no eixo y, calcula a distância `d`. Caso esta distância for menor que 0.5, significa que a coordenada aleatória está dentro do círculo. A área estimada `area_estimate` é calculada como a razão entre a quantidade de vezes que a coordenada foi considerada dentro do círculo em relação a quantidade total de pontos gerados.

In [114]:
import numpy as np
from random import uniform

def calculate_pi(n=1_000_000):
    count = 0
    for i in range(n):
        u, v = uniform(0, 1), uniform(0, 1)
        d = np.sqrt((u - 0.5)**2 + (v - 0.5)**2)
        if d < 0.5:
            count += 1

    area_estimate = count / n
    return area_estimate * 4  # dividing by radius**2

Este código, quando executado com o valor de `n` padrão, obtemos o seguinte tempo de execução.

In [115]:
%timeit calculate_pi()

2.13 s ± 829 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### **(Tarefa)** Empregue o decorador `@jit` para acelerar `calculate_pi`

In [116]:
import numba
from numba import jit
@jit(nopython=True)
def calculate_pi_jit(n=1_000_000):
    count = 0
    for i in range(n):
        u, v = uniform(0, 1), uniform(0, 1)
        d = np.sqrt((u - 0.5)**2 + (v - 0.5)**2)
        if d < 0.5:
            count += 1

    area_estimate = count / n
    return area_estimate * 4  # dividing by radius**2

Para fins de exemplo, abaixo a medição de tempo obtida pelo professor (substitua pela sua implementação).

In [117]:
%timeit calculate_pi_jit()

20.6 ms ± 6.02 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Parte 2 - JIT para GPU

Para este exercício, vamos abordar o funcionamento de reduções com Numba CUDA e como gerar números aleatórios na GPU.

### Redução em GPU

Para tal, primeiro vemos a redução de um vetor para um único valor, por exemplo, utilizando `np.sum`.

In [118]:
import numpy as np
v = np.arange(1,1_000_000)
np.sum(v)

499999500000

Para fazer o mesmo em GPU, precisamos utilizar o decorador `@cuda.reduce` em uma função de aridade dupla (com dois argumentos), assim:

In [119]:
from numba import cuda
@cuda.reduce
def sum_reduce(a, b):
    return a + b

Vamos agora testá-la:
1. Gerar um vetor `v` de elementos
2. Copiar os dados para o device, com `cuda.to_device()`
3. Invocar o kernel com a referência dos dados já no device.

In [120]:
v = np.arange(1, 1_000_000)   #1
dev_v = cuda.to_device(v)     #2
sum_reduce(dev_v)             #3

/usr/local/lib/python3.7/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 64 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.7/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


499999500000

### Geração de números aleatórios em GPU

Ao empregar a geração de números aleatórios na GPU, é importante garantir que cada CUDA thread tenha o seu próprio estado, de uma maneira que cada CUDA thread gere sequências independentes, sem intersecção. O módulo `numba.cuda.random` fornece toda a funcionalidade para colocar isso em prática, inclusive para gerar números uniformemente distribuídos (necessário para o cálculo de pi). Vamos revisar esses elementos agora.

Primeiro, vamos importar as funções que importam. A função `create_xoroshiro128p_states` gera um arranjo de estados independentes a serem utilizados no device, enquanto que `xoroshiro128p_uniform_float32` gera números uniformemente distribuídos em um intervalo, e recebe como entrada o arranjo de estados. Esta deve ser chamada no código executado no device.

In [121]:
from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_uniform_float32

Vamos criar na memória do device um arranjo com os estados independentes. Neste exemplo, estamos criando 128 estados. Em uso prático, deve haver um estado para cada CUDA thread que será lançada.

In [122]:
quantidade_de_estados = 128
estados = create_xoroshiro128p_states (quantidade_de_estados, seed = 1)
estados

A função `xoroshiro128p_uniform_float32`, chamada no código executado no device, retorna um número no intervalo [0, 1). Ela recebe os estados o identificador única da CUDA thread que está a invocando.

A função abaixo `gera_numeros_aleatorios_na_GPU` é um gerador paralelo de números aleatórios usando Numba em GPU.

In [123]:
from numba import cuda
@cuda.jit
def gera_numeros_aleatorios_na_GPU(estados, out):
    tid = cuda.grid(1) # identificador único da thread
    x = xoroshiro128p_uniform_float32(estados, tid)
    out[tid] = x

Vamos gerar `N` números aleatórios, em blocos com 64 threads.

In [124]:
N = 128*1000                      # garantir que N é múltipl de 64
threads_per_block = 64
blocks = int(N/threads_per_block) # deve ser um número inteiro

Criar os estados.

In [125]:
estados = create_xoroshiro128p_states(N, seed=1)

Agora, preparar a variável de saída `out`, copiá-la para a GPU com `to_device`, invocar o kernel e repatriar os dados para a RAM.

In [126]:
import numpy as np
out = np.zeros(N, dtype=np.float32)
out_dev = cuda.to_device(out)
gera_numeros_aleatorios_na_GPU[blocks, threads_per_block](estados, out_dev)
saida = out_dev.copy_to_host()
saida

array([0.13312314, 0.87634873, 0.8710909 , ..., 0.2169639 , 0.49467936,
       0.19529185], dtype=float32)

Agora vamos à tarefa.

In [127]:
saida.size

128000

### **(Tarefa)** Empregue o decorador `@cuda.jit` para acelerar `calculate_pi`

O objetivo desta tarefa é realizar o cálculo de pi na GPU, empregando o decorador `@cuda.jit`.

Para realizar esta tarefa, uma estratégia possível é fazer com que cada thread gere coordenadas de maneira aleatória (uma ou mais vezes), verificando se cada coordenada gerada faz parte do círculo unitário e contabilizando aquelas que fazem parte. Como cada thread sabe quantas coordenadas foram geradas, ela consegue calcular sua aproximação local para o valor de pi (da mesma forma que o código disponibilizado no início deste caderno de anotações). No final, os resultados destas aproximações estarão em arranjo na memória da GPU. Para terminar, podes acumular estas aproximações fazendo um processo de redução na GPU.

In [128]:
from numba import cuda
import numpy as np
import math

@cuda.jit
def gera_coordenadas_GPU(estados, out):       
    tid = cuda.grid(1) # identificador único da thread
    u = xoroshiro128p_uniform_float32(estados, tid)
    v = xoroshiro128p_uniform_float32(estados, tid)    
    d = math.sqrt((u - 0.5)**2 + (v - 0.5)**2)
    if d < 0.5:   
      out[tid] = 1
    else:
      out[tid] = 0  



In [129]:

import numpy as np

def calculate_pi_jitcuda():  
  estados = create_xoroshiro128p_states(N, seed=1)
  out = np.zeros(N, dtype=np.float32)
  out_dev = cuda.to_device(out)
  gera_coordenadas_GPU[blocks, threads_per_block](estados, out_dev)
  saida = out_dev.copy_to_host()
  saida
  # saida.size

  # Somatório e cálculo final
  dev_v = cuda.to_device(saida)
  area_estimate = sum_reduce(dev_v) / saida.size
  area_estimate
  area_estimate * 4

In [130]:
%timeit calculate_pi_jitcuda()

/usr/local/lib/python3.7/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 64 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.7/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


35 ms ± 1.92 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Questionamentos finais

1. A versão em GPU ficou mais rápida que o código (com ou sem JIT) CPU observado acima?


Resultados
Código comum:

2.1 s ± 864 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Código JIT:

14.9 ms ± 2.61 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Código JIT CUDA:

27.5 ns ± 0.36 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)

Sim. O resultado com GPU foi muito superior.

2. Na versão GPU, observa-se alguma diferença de desempenho entre a quantidade de threads por bloco?

Testamos com valores de blocos variados: 16, 32, 128, 256, 512.

Não encontramos variação significativa.

In [131]:
N = 128*1000                      # garantir que N é múltipl de 64
threads_per_block = 64
blocks = int(N/threads_per_block) # deve ser um número inteiro

%timeit calculate_pi_jitcuda

32.7 ns ± 10.7 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [132]:
N = 128*1000                      # garantir que N é múltipl de 64
threads_per_block = 32
blocks = int(N/threads_per_block) # deve ser um número inteiro

%timeit calculate_pi_jitcuda

27.7 ns ± 5.95 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [133]:
N = 128*1000                      # garantir que N é múltipl de 64
threads_per_block = 16
blocks = int(N/threads_per_block) # deve ser um número inteiro

%timeit calculate_pi_jitcuda

34.2 ns ± 7.43 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [134]:
N = 128*1000                      # garantir que N é múltipl de 64
threads_per_block = 128
blocks = int(N/threads_per_block) # deve ser um número inteiro

%timeit calculate_pi_jitcuda

24.7 ns ± 0.383 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [135]:
N = 128*1000                      # garantir que N é múltipl de 64
threads_per_block = 256
blocks = int(N/threads_per_block) # deve ser um número inteiro

%timeit calculate_pi_jitcuda

24.7 ns ± 0.419 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [144]:
N = 128*1000                      # garantir que N é múltipl de 64
threads_per_block = 512
blocks = int(N/threads_per_block) # deve ser um número inteiro

%timeit calculate_pi_jitcuda

24.4 ns ± 0.501 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
